In [9]:
# Import modules
import os
import pandas as pd
import rdkit
# from rdkit.Chem import Draw
# from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize

# Expand to see all columns
pd.set_option('display.max_columns', None)

# Suppress RDKit Output
rdkit.RDLogger.DisableLog('rdApp.*')

# Print versions
print(f"Pandas Version: {pd.__version__}")
print(f"RDKit Version: {rdkit.__version__}")

Pandas Version: 2.2.3
RDKit Version: 2024.09.6


In [10]:
# Paths
path_data = os.path.realpath("../data")
input_file = "chembl_approved_small_molecule_drugs.csv"
output_file = "chembl_approved_small_molecule_drugs_rdkit-profiled.csv"
input_path = os.path.join(path_data, input_file)
output_path = os.path.join(path_data, output_file)

# Load CSV
df = pd.read_csv(input_path)
df.head()

,chembl_molecule_id,smiles,molregno,pref_name,max_phase,therapeutic_flag,dosed_ingredient,structure_type,chebi_par_id,molecule_type,first_approval,oral,parenteral,topical,black_box_warning,natural_product,first_in_class,chirality,prodrug,inorganic_flag,usan_year,availability_type,usan_stem,polymer_flag,usan_substem,usan_stem_definition,indication_class,withdrawn_flag,chemical_probe,orphan,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,np_likeness_score
0,CHEMBL1200542,CC(=O)OCC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)C...,674493,DESOXYCORTICOSTERONE ACETATE,4.0,1,1,MOL,34671.0,Small molecule,1939.0,0,1,0,0,1,0,1,0,0,NaN,0.0,-cort-; -ster-; -terone,0,-cort-; -ster-; -terone,"cortisone derivatives; steroids (androgens, an...",Adrenocortical Steroid (salt-regulating),0,0,0,372.51,4.27,4.0,0.0,60.44,3.0,N,0.0,NaN,NaN,3.77,3.77,NEUTRAL,372.51,0.0,27.0,0.69,372.2301,C23H32O4,4.0,0.0,0.0,1.96
1,CHEMBL1200728,Cl.N=C(N)N,674679,GUANIDINE HYDROCHLORIDE,4.0,1,1,MOL,32735.0,Small molecule,1939.0,1,0,0,0,0,0,2,0,0,NaN,1.0,guan-,0,guan-,antihypertensives (guanidine derivatives),NaN,0,0,0,59.07,-1.16,1.0,3.0,75.89,0.0,N,0.0,NaN,12.55,-1.24,-3.65,BASE,95.53,0.0,4.0,0.24,59.0483,CH6ClN3,3.0,5.0,0.0,0.32
2,CHEMBL1200982,CCC(C)C1(CC)C(=O)[N-]C(=O)NC1=O.[Na+],674933,BUTABARBITAL SODIUM,4.0,1,1,MOL,NaN,Small molecule,1939.0,1,0,0,0,0,0,0,0,0,NaN,1.0,-barb-,0,-barb-,barbituric acid derivatives,Sedative-Hypnotic,0,0,0,212.25,0.79,3.0,2.0,75.27,3.0,N,0.0,7.48,NaN,1.45,1.19,NEUTRAL,234.23,0.0,15.0,0.68,212.1161,C10H15N2NaO3,5.0,2.0,0.0,0.32
3,CHEMBL3989520,NCCc1c[nH]cn1.O=P(O)(O)O.O=P(O)(O)O,2197391,HISTAMINE PHOSPHATE,4.0,1,1,MOL,NaN,Small molecule,1939.0,0,1,0,1,0,0,2,0,0,NaN,0.0,NaN,0,NaN,NaN,NaN,0,0,0,111.15,-0.09,2.0,2.0,54.70,2.0,Y,0.0,NaN,9.58,-0.70,-2.85,BASE,307.14,1.0,8.0,0.56,111.0796,C5H15N3O8P2,3.0,3.0,0.0,0.00
4,CHEMBL449,CCC(C)C1(CC)C(=O)NC(=O)NC1=O,2393,BUTABARBITAL,4.0,1,0,MOL,3228.0,Small molecule,1939.0,1,0,0,0,1,0,0,0,0,NaN,1.0,-barb-,0,-barb-,barbituric acid derivatives,Sedative-Hypnotic,0,0,0,212.25,0.79,3.0,2.0,75.27,3.0,N,0.0,7.48,NaN,1.45,1.19,NEUTRAL,212.25,0.0,15.0,0.68,212.1161,C10H16N2O3,5.0,2.0,0.0,0.32


In [11]:
largest_frag_app = rdMolStandardize.LargestFragmentChooser()
uncharge_app = rdMolStandardize.Uncharger()
mols = []
skipped_indices = []
for ii, smi in enumerate(df["smiles"]):
    try:
        mol = rdkit.Chem.MolFromSmiles(
            smi,
            sanitize = False
        )
        assert mol is not None
    except Exception as e:
        skipped_indices.append(ii)
        if pd.notna(smi):
            print(f"Error processing SMILES at index {ii}: {smi}")
            print(f"Exception: {e}")
        continue
    try:
        rdkit.Chem.SanitizeMol(mol)
        largest_frag_app.chooseInPlace(mol)
        uncharge_app.unchargeInPlace(mol)
    except Exception as e:
        skipped_indices.append(ii)
        print(f"Error processing molecule at index {ii}")
        print(f"Exception: {e}")
        continue
    descriptors = {
        "idx": ii,
        "SMILES": rdkit.Chem.MolToSmiles(mol),
    }
    for descriptor, fxn in Descriptors._descList:
        try:
            value = fxn(mol)
        except Exception as e:
            # print(f"Error calculating {descriptor} at index {ii}")
            # print(f"Exception: {e}")
            value = None
        descriptors[f"rdkit_{descriptor}"] = value
    mols.append(descriptors)

print(f"{len(mols)}/{df.shape[0]} molecules processed!")
print(f"{df.shape[0]-len(mols)}/{df.shape[0]} molecules skipped!")

3382/3517 molecules processed!
135/3517 molecules skipped!


In [12]:
column_update = { cc: f"chembl_{cc}" for cc in df.columns[1:] }
df = df.rename(columns=column_update)
# for mol_dict in mols:
#     idx = mol_dict["idx"]
#     for key, value in mol_dict.items():
#         if key != "idx":
#             col = key if key == "SMILES" else key.lower()
#             df.loc[idx, col] = value
# Convert list of dictionaries to a DataFrame
mol_df = pd.DataFrame(mols)
mol_df.set_index("idx", inplace=True)

mol_df.columns = [ col if col == "SMILES" else col.lower() 
                   for col in mol_df.columns ]

df = pd.concat([df, mol_df], axis=1)
cols = df.columns.tolist()
cols.remove('SMILES')
cols.insert(0, 'SMILES')
df = df[cols]

# Display results
df.head()

,SMILES,chembl_molecule_id,chembl_smiles,chembl_molregno,chembl_pref_name,chembl_max_phase,chembl_therapeutic_flag,chembl_dosed_ingredient,chembl_structure_type,chembl_chebi_par_id,chembl_molecule_type,chembl_first_approval,chembl_oral,chembl_parenteral,chembl_topical,chembl_black_box_warning,chembl_natural_product,chembl_first_in_class,chembl_chirality,chembl_prodrug,chembl_inorganic_flag,chembl_usan_year,chembl_availability_type,chembl_usan_stem,chembl_polymer_flag,chembl_usan_substem,chembl_usan_stem_definition,chembl_indication_class,chembl_withdrawn_flag,chembl_chemical_probe,chembl_orphan,chembl_mw_freebase,chembl_alogp,chembl_hba,chembl_hbd,chembl_psa,chembl_rtb,chembl_ro3_pass,chembl_num_ro5_violations,chembl_cx_most_apka,chembl_cx_most_bpka,chembl_cx_logp,chembl_cx_logd,chembl_molecular_species,chembl_full_mwt,chembl_aromatic_rings,chembl_heavy_atoms,chembl_qed_weighted,chembl_mw_monoisotopic,chembl_full_molformula,chembl_hba_lipinski,chembl_hbd_lipinski,chembl_num_lipinski_ro5_violations,chembl_np_likeness_score,rdkit_maxabsestateindex,rdkit_maxestateindex,rdkit_minabsestateindex,rdkit_minestateindex,rdkit_qed,rdkit_sps,rdkit_molwt,rdkit_heavyatommolwt,rdkit_exactmolwt,rdkit_numvalenceelectrons,rdkit_numradicalelectrons,rdkit_maxpartialcharge,rdkit_minpartialcharge,rdkit_maxabspartialcharge,rdkit_minabspartialcharge,rdkit_fpdensitymorgan1,rdkit_fpdensitymorgan2,rdkit_fpdensitymorgan3,rdkit_bcut2d_mwhi,rdkit_bcut2d_mwlow,rdkit_bcut2d_chghi,rdkit_bcut2d_chglo,rdkit_bcut2d_logphi,rdkit_bcut2d_logplow,rdkit_bcut2d_mrhi,rdkit_bcut2d_mrlow,rdkit_avgipc,rdkit_balabanj,rdkit_bertzct,rdkit_chi0,rdkit_chi0n,rdkit_chi0v,rdkit_chi1,rdkit_chi1n,rdkit_chi1v,rdkit_chi2n,rdkit_chi2v,rdkit_chi3n,rdkit_chi3v,rdkit_chi4n,rdkit_chi4v,rdkit_hallkieralpha,rdkit_ipc,rdkit_kappa1,rdkit_kappa2,rdkit_kappa3,rdkit_labuteasa,rdkit_peoe_vsa1,rdkit_peoe_vsa10,rdkit_peoe_vsa11,rdkit_peoe_vsa12,rdkit_peoe_vsa13,rdkit_peoe_vsa14,rdkit_peoe_vsa2,rdkit_peoe_vsa3,rdkit_peoe_vsa4,rdkit_peoe_vsa5,rdkit_peoe_vsa6,rdkit_peoe_vsa7,rdkit_peoe_vsa8,rdkit_peoe_vsa9,rdkit_smr_vsa1,rdkit_smr_vsa10,rdkit_smr_vsa2,rdkit_smr_vsa3,rdkit_smr_vsa4,rdkit_smr_vsa5,rdkit_smr_vsa6,rdkit_smr_vsa7,rdkit_smr_vsa8,rdkit_smr_vsa9,rdkit_slogp_vsa1,rdkit_slogp_vsa10,rdkit_slogp_vsa11,rdkit_slogp_vsa12,rdkit_slogp_vsa2,rdkit_slogp_vsa3,rdkit_slogp_vsa4,rdkit_slogp_vsa5,rdkit_slogp_vsa6,rdkit_slogp_vsa7,rdkit_slogp_vsa8,rdkit_slogp_vsa9,rdkit_tpsa,rdkit_estate_vsa1,rdkit_estate_vsa10,rdkit_estate_vsa11,rdkit_estate_vsa2,rdkit_estate_vsa3,rdkit_estate_vsa4,rdkit_estate_vsa5,rdkit_estate_vsa6,rdkit_estate_vsa7,rdkit_estate_vsa8,rdkit_estate_vsa9,rdkit_vsa_estate1,rdkit_vsa_estate10,rdkit_vsa_estate2,rdkit_vsa_estate3,rdkit_vsa_estate4,rdkit_vsa_estate5,rdkit_vsa_estate6,rdkit_vsa_estate7,rdkit_vsa_estate8,rdkit_vsa_estate9,rdkit_fractioncsp3,rdkit_heavyatomcount,rdkit_nhohcount,rdkit_nocount,rdkit_numaliphaticcarbocycles,rdkit_numaliphaticheterocycles,rdkit_numaliphaticrings,rdkit_numamidebonds,rdkit_numaromaticcarbocycles,rdkit_numaromaticheterocycles,rdkit_numaromaticrings,rdkit_numatomstereocenters,rdkit_numbridgeheadatoms,rdkit_numhacceptors,rdkit_numhdonors,rdkit_numheteroatoms,rdkit_numheterocycles,rdkit_numrotatablebonds,rdkit_numsaturatedcarbocycles,rdkit_numsaturatedheterocycles,rdkit_numsaturatedrings,rdkit_numspiroatoms,rdkit_numunspecifiedatomstereocenters,rdkit_phi,rdkit_ringcount,rdkit_mollogp,rdkit_molmr,rdkit_fr_al_coo,rdkit_fr_al_oh,rdkit_fr_al_oh_notert,rdkit_fr_arn,rdkit_fr_ar_coo,rdkit_fr_ar_n,rdkit_fr_ar_nh,rdkit_fr_ar_oh,rdkit_fr_coo,rdkit_fr_coo2,rdkit_fr_c_o,rdkit_fr_c_o_nocoo,rdkit_fr_c_s,rdkit_fr_hoccn,rdkit_fr_imine,rdkit_fr_nh0,rdkit_fr_nh1,rdkit_fr_nh2,rdkit_fr_n_o,rdkit_fr_ndealkylation1,rdkit_fr_ndealkylation2,rdkit_fr_nhpyrrole,rdkit_fr_sh,rdkit_fr_aldehyde,rdkit_fr_alkyl_carbamate,rdkit_fr_alkyl_halide,rdkit_fr_allylic_oxid,rdkit_fr_amide,rdkit_fr_amidine,rdkit_fr_aniline,rdkit_fr_aryl_methyl,rdkit_fr_azide,rdkit_fr_azo,rdkit_fr_barbitur,rdkit_fr_ben